In [9]:
import pprint
import json
import copy

pp = pprint.PrettyPrinter(indent=4)

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

import os
os.chdir("/content/gdrive/MyDrive/CS542/cs542-autocast/autocast/cs542")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# data = json.load(open(f'autocast_cc_news_retrieved.json'))

In [ ]:
# negated_questions = json.load(open(f'negated_tf_questions.json'))
# autocast_questions = json.load(open(f'autocast_questions.json'))
# qid_to_question = {q['id']: q for q in autocast_questions}
# qid_to_negation = {q['id']: q for q in negated_questions}
# all_questions = []
# for q in data:
#     q['qtype'] = qid_to_question[q['question_id']]['qtype']
#     if q['question_id'] in qid_to_negation:
#         negated_q = copy.deepcopy(q)
#         negated_q['question'] = qid_to_negation[q['question_id']]['negated']
#         for day in negated_q['targets']:
#             day['target'] = 1 - float(day['target']) # flip the forecast probabilities
#         if q['answers'][0] == 'yes':
#             negated_q['answers'] = ['no']
#         else:
#             negated_q['answers'] = ['yes']
#         all_questions.append(negated_q)
        
#     all_questions.append(q)

In [11]:
negated_questions = json.load(open(f'huggingface_cache/negated_tf_questions.json'))
autocast_questions = json.load(open(f'huggingface_cache/autocast_questions.json'))
qid_to_question = {q['id']: q for q in autocast_questions}
qid_to_negation = {q['id']: q for q in negated_questions}
temp_train_qs, temp_test_qs = [],[]
for q in autocast_questions:
    # if q['close_time'] < '2021-05-11':
    #     continue
    if q['answer'] is None:
        continue
    # q['qtype'] = qid_to_question[q['question_id']]['qtype']
    # Remove useless parts
    simplfied_q = {"question": q["question"], "id": q["id"], "background": q["background"], "answer": q["answer"], "choices": q["choices"], "qtype": q["qtype"]}
    if q['id'] in qid_to_negation:
        negated_q = copy.deepcopy(simplfied_q)
        negated_q['question'] = qid_to_negation[q['id']]['negated']
        # for day in negated_q['targets']:
        #     day['target'] = 1 - float(day['target']) # flip the forecast probabilities
        if q['answer'] == 'yes':
            negated_q['answer'] = 'no'
        else:
            negated_q['answer'] = 'yes'
        if q['close_time'] < '2021-05-11':
            temp_train_qs.append(negated_q)
        else:
            temp_test_qs.append(negated_q)
        # temp_test_qs.append(negated_q)
        
    if q['close_time'] < '2021-05-11':
        temp_train_qs.append(simplfied_q)
    else:
        temp_test_qs.append(simplfied_q)



In [12]:
print(len(temp_train_qs), len(temp_test_qs))

1590 4161


In [13]:
with open(f'temporal_train.json', "w", encoding='utf-8') as writer:
    writer.write(json.dumps(temp_train_qs, indent=4, ensure_ascii=False) + "\n")
with open(f'temporal_test.json', "w", encoding='utf-8') as writer:
    writer.write(json.dumps(temp_test_qs, indent=4, ensure_ascii=False) + "\n")

## Convert to static

In [ ]:
method = 'bm25ce'
descending = method != 'dpr'

In [ ]:
static_data = []
for question in all_questions:
    new_question = copy.deepcopy(question)
    del new_question['targets']

    all_ctxs = []
    all_scores = []
    for target in question['targets']:
        if target['ctxs']:
            all_ctxs.extend(target['ctxs'])
            all_scores.extend([float(ctxs['score']) for ctxs in target['ctxs']])
    sorted_idx = [x for _, x in sorted(zip(all_scores, range(len(all_scores))), reverse=descending)]
    new_question['ctxs'] = [all_ctxs[i] for i in sorted_idx][:15]
    
    static_data.append(new_question)

KeyError: 'targets'

In [ ]:
c1, c2 = 0,0
static_train_qs, static_test_qs = [],[]
for d in static_data:
    if d['question_expiry'] < '2021-05-11':
        c1 += 1
        static_train_qs.append(d)
    else:
        static_test_qs.append(d)
        c2 += 1
c1, c2

(4387, 1364)

In [ ]:
with open(f'static_train.json', "w", encoding='utf-8') as writer:
    writer.write(json.dumps(static_train_qs, indent=4, ensure_ascii=False) + "\n")
with open(f'static_test.json', "w", encoding='utf-8') as writer:
    writer.write(json.dumps(static_test_qs, indent=4, ensure_ascii=False) + "\n")